# LSTM Bot

## Project Overview

In this project, you will build a chatbot that can converse with you at the command line. The chatbot will use a Sequence to Sequence text generation architecture with an LSTM as it's memory unit. You will also learn to use pretrained word embeddings to improve the performance of the model. At the conclusion of the project, you will be able to show your chatbot to potential employers.

Additionally, you have the option to use pretrained word embeddings in your model. We have loaded Brown Embeddings from Gensim in the starter code below. You can compare the performance of your model with pre-trained embeddings against a model without the embeddings.



---



A sequence to sequence model (Seq2Seq) has two components:
- An Encoder consisting of an embedding layer and LSTM unit.
- A Decoder consisting of an embedding layer, LSTM unit, and linear output unit.

The Seq2Seq model works by accepting an input into the Encoder, passing the hidden state from the Encoder to the Decoder, which the Decoder uses to output a series of token predictions.

## Dependencies

- Pytorch
- Numpy
- Pandas
- NLTK
- Gzip
- Gensim


Please choose a dataset from the Torchtext website. We recommend looking at the Squad dataset first. Here is a link to the website where you can view your options:

- https://pytorch.org/text/stable/datasets.html





In [ ]:
#!pip uninstall torch torchvision torchaudio -y

In [ ]:
#!pip install torch==1.7.1+cu110 torchvision==0.8.2+cu110 torchaudio==0.7.2 -f https://download.pytorch.org/whl/torch_stable.html

In [ ]:
#!pip install torch==1.7.1+cu110 torchvision==0.8.2+cu110 torchaudio==0.7.2 -f https://download.pytorch.org/whl/torch_stable.html

In [1]:
!pip install -U pip setuptools wheel
!pip install -U spacy
!python -m spacy download en_core_web_sm

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.8/12.8 MB 51.9 MB/s eta 0:00:0000:0100:01
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_sm')


In [2]:
!pip show torch

Name: torch
Version: 1.11.0
Summary: Tensors and Dynamic neural networks in Python with strong GPU acceleration
Home-page: https://pytorch.org/
Author: PyTorch Team
Author-email: packages@pytorch.org
License: BSD-3
Location: /opt/conda/lib/python3.7/site-packages
Requires: typing-extensions
Required-by: torchtext, torchvision


In [3]:
import torch;
torch.__version__

'1.11.0+cu102'

In [4]:
import pandas as pd
import json
import string

In [5]:
import gensim
import nltk
import numpy as np
import pandas as pd
import gzip
import torch
from nltk.corpus import brown
from nltk.tokenize import word_tokenize
from pandas.io.json import json_normalize 
from torch.utils.data import TensorDataset, DataLoader

nltk.download('brown')
nltk.download('punkt')

# Output, save, and load brown embeddings

model_gensim = gensim.models.Word2Vec(brown.sents())
model_gensim.save('brown.embedding')

w2v = gensim.models.Word2Vec.load('brown.embedding')

def clean_text(x):
    if len(x) > 0:
        return x[0]["text"]

def loadDF(path):
    
    df = pd.read_json(path) 
    data = pd.json_normalize(data = df['data'],
                            record_path =['paragraphs', 'qas'])
    data = data.drop('id', axis=1)
    data = data.drop('is_impossible', axis=1)
    data = data.drop('plausible_answers', axis=1)
    data['answers'] = data['answers'].apply(clean_text)
    
    return data



#, dev_test
def prepare_text(df_train):
    

    
    return df_train #, dev_test



def train_test_split(SRC, TRG):
    
    '''
    Input: SRC, our list of questions from the dataset
            TRG, our list of responses from the dataset

    Output: Training and test datasets for SRC & TRG

    '''
    
    return SRC_train_dataset, SRC_test_dataset, TRG_train_dataset, TRG_test_dataset


[nltk_data] Downloading package brown to /root/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [6]:
df_train = loadDF('./train-v2.0.json')
dev_test = loadDF('./dev-v2.0.json')

In [7]:
df_train.dropna()
dev_test.dropna()
df_train = df_train[df_train['answers'].notna()]
dev_test = dev_test[dev_test['answers'].notna()]
df_train = df_train[df_train['question'].notna()]
dev_test = dev_test[dev_test['question'].notna()]

In [8]:
pd.set_option('display.max_rows', None)

In [9]:
df_train = df_train[0:5000]
dev_test = dev_test[0:5000]
df_train.head()

,question,answers
0,When did Beyonce start becoming popular?,in the late 1990s
1,What areas did Beyonce compete in when she was...,singing and dancing
2,When did Beyonce leave Destiny's Child and bec...,2003
3,In what city and state did Beyonce grow up?,"Houston, Texas"
4,In which decade did Beyonce become famous?,late 1990s


In [10]:
SOS_token = 55000
EOS_token = 55001
PAD_IDX = 55002
word2VectorDict = w2v.wv.key_to_index 

class Lang:
    def __init__(self, name):
        self.name = name
        self.word2index = {"SOS": 55000, "EOS": 55001, "PAD_IDX": 55002}
        self.word2count = {}
        self.index2word = {55000: "SOS", 55001: "EOS", 55002: "PAD_IDX"}
        self.n_words = 3  

    def addSentence(self, sentence):
        for word in sentence.split(' '):
            self.addWord(word)
            

    def addWord(self, word):
        if word not in self.word2index:
                if word in word2VectorDict:
                    indexOfWord = word2VectorDict[word]
                    self.word2index[word] = indexOfWord
                    self.index2word[indexOfWord] = word
                    self.n_words += 1
                else:
                    self.word2index[word] = self.n_words
                    self.index2word[self.n_words] = word
                    self.n_words += 1
                self.word2count[word] = 1
        else:
            self.word2count[word] += 1
            
lang = Lang(name='alltokens')
Questionlang = Lang(name='allQuestiontokens')
Answerlang = Lang(name='allAnswertokens')

In [142]:
'''Reference:-  https://pytorch.org/tutorials/beginner/torchtext_translation_tutorial.html
Some methods are taken from this tutorial'''

'Reference:-  https://pytorch.org/tutorials/beginner/torchtext_translation_tutorial.html\nSome methods are taken from this tutorial'

In [11]:
import torchtext
import torch
from torchtext.data.utils import get_tokenizer
from collections import Counter
from torchtext.vocab import vocab
import spacy

en_nlp = spacy.load("en_core_web_sm")


en_tokenizer = get_tokenizer('spacy', language='en_core_web_sm')

def build_vocab(df, caption, tokenizer):
    default_index = 0
    counter = Counter()
    for index, row, in df.iterrows():
        counter.update(tokenizer(row[caption]))
    v2 = vocab(counter, specials=['<unk>', '<pad>', '<bos>', '<eos>'])
    v2.set_default_index(default_index)
    v2.set_default_index(v2['<unk>'])
    #for index, row, in df.iterrows():
    #    lang.addWord(tokenizer(row[caption]))        
    return v2

question_vocab = build_vocab(df_train, "question", en_tokenizer)
answer_vocab = build_vocab(df_train, "answers", en_tokenizer)

def data_process(df):
    
    data = []
    for index, (question, answers) in df.iterrows():
        question_tensor_ = torch.tensor([question_vocab[token] for token in en_tokenizer(question)],
                            dtype=torch.long)
        answer_tensor_ = torch.tensor([answer_vocab[token] for token in en_tokenizer(answers)],
                            dtype=torch.long)
        data.append((question_tensor_, answer_tensor_))
    return data

train_data = data_process(df_train)
val_data = data_process(dev_test)

In [12]:
train_data[0]

(tensor([ 4,  5,  6,  7,  8,  9, 10]), tensor([4, 5, 6, 7]))

In [13]:
val_data[0]

(tensor([  29,   30, 1300,  112,    0,  225,   10]), tensor([798]))

In [14]:
len(train_data)
len(val_data)

5000

In [141]:
'''Reference:-  https://pytorch.org/tutorials/beginner/torchtext_translation_tutorial.html
Some methods generate_batch are taken from this tutorial'''

'Reference:-  https://pytorch.org/tutorials/beginner/torchtext_translation_tutorial.html\nSome methods generate_batch are taken from this tutorial'

In [15]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
BATCH_SIZE = 128
PAD_IDX = question_vocab['<pad>']
BOS_IDX = question_vocab['<bos>']
EOS_IDX = question_vocab['<eos>']


from torch.nn.utils.rnn import pad_sequence
from torch.utils.data import DataLoader

def generate_batch(data_batch):
    de_batch, en_batch = [], []
    for (de_item, en_item) in data_batch:
        de_batch.append(torch.cat([torch.tensor([BOS_IDX]), de_item, torch.tensor([EOS_IDX])], dim=0))
        en_batch.append(torch.cat([torch.tensor([BOS_IDX]), en_item, torch.tensor([EOS_IDX])], dim=0))
        de = pad_sequence(de_batch, padding_value=PAD_IDX)
        en = pad_sequence(en_batch, padding_value=PAD_IDX)
    return de, en

train_iter = DataLoader(train_data, batch_size=BATCH_SIZE,
                        shuffle=True, collate_fn=generate_batch)
valid_iter = DataLoader(val_data, batch_size=BATCH_SIZE,
                        shuffle=True, collate_fn=generate_batch)
#test_iter = DataLoader(test_data, batch_size=BATCH_SIZE,
#                       shuffle=True, collate_fn=generate_batch)

cuda


In [16]:
print(EOS_IDX)

3


In [17]:
#df_train.head(5000)

In [18]:
for _, (src, trg) in enumerate(train_iter):
    print(trg)

tensor([[   2,    2,    2,  ...,    2,    2,    2],
        [3683,  734, 2415,  ...,    5,  115, 1053],
        [   3, 2683, 2416,  ...,  301,  116,    3],
        ...,
        [   1,    1,    1,  ...,    1,    1,    1],
        [   1,    1,    1,  ...,    1,    1,    1],
        [   1,    1,    1,  ...,    1,    1,    1]])
tensor([[   2,    2,    2,  ...,    2,    2,    2],
        [ 809, 2272,   43,  ...,    5, 2295,   55],
        [ 810, 2273,    3,  ..., 1400, 2296,   56],
        ...,
        [   1,    1,    1,  ...,    1,    1,    1],
        [   1,    1,    1,  ...,    1,    1,    1],
        [   1,    1,    1,  ...,    1,    1,    1]])
tensor([[   2,    2,    2,  ...,    2,    2,    2],
        [ 121, 1938, 4281,  ...,  148,  495,   35],
        [ 840,   13, 4324,  ...,    3,    3,    3],
        ...,
        [   1,    1,    1,  ...,    1,    1,    1],
        [   1,    1,    1,  ...,    1,    1,    1],
        [   1,    1,    1,  ...,    1,    1,    1]])
tensor([[   2,    2,  

In [140]:
'''bentrevett/pytorch-seq2seq
https://github.com/bentrevett/pytorch-seq2seq/blob/master/1%20-%20Sequence%20to%20Sequence%20Learning%20with%20Neural%20Networks.ipynb
Some Encoder/decoder/seqtoSeq methods are taken from this tutorial
'''


'bentrevett/pytorch-seq2seq\nhttps://github.com/bentrevett/pytorch-seq2seq/blob/master/1%20-%20Sequence%20to%20Sequence%20Learning%20with%20Neural%20Networks.ipynb\nSome Encoder/decoder/seqtoSeq methods are taken from this tutorial\n'

In [19]:
import random
from typing import Tuple

import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch import Tensor

class Encoder(nn.Module):
    def __init__(self,
                 input_dim: int,
                 emb_dim: int,
                 hid_dim: int,
                 n_layers: int,
                 dropout: float):
        super().__init__()

        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(input_dim, emb_dim)
        
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src len, batch size]
        
        embedded = self.dropout(self.embedding(src))
        
        #embedded = [src len, batch size, emb dim]
        
        outputs, (hidden, cell) = self.rnn(embedded)
        
        #outputs = [src len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #outputs are always from the top hidden layer
        
        return hidden, cell

In [20]:
class Decoder(nn.Module):
    def __init__(self,
                 output_dim: int,
                 emb_dim: int,
                 hid_dim: int,
                 n_layers: int,
                 dropout: int):
        super().__init__()

        self.output_dim = output_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        
        self.fc_out = nn.Linear(hid_dim, output_dim)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, input, hidden, cell):
              
        input = input.unsqueeze(0)
               
        embedded = self.dropout(self.embedding(input))
 
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        
        prediction = self.fc_out(output.squeeze(0))
                
        return prediction, hidden, cell

In [21]:
class Seq2Seq(nn.Module):
    def __init__(self,
                 encoder: nn.Module,
                 decoder: nn.Module,
                 device: torch.device):
        super().__init__()

        self.encoder = encoder
        self.decoder = decoder
        self.device = device

    def forward(self,
                src: Tensor,
                trg: Tensor,
                teacher_forcing_ratio: float = 0.5) -> Tensor:

        batch_size = src.shape[1]
        max_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim

        outputs = torch.zeros(max_len, batch_size, trg_vocab_size).to(self.device)

        hidden, cell = self.encoder(src)

        # first input to the decoder is the <sos> token
        output = trg[0,:]

        for t in range(1, max_len):
            output, hidden, cell = self.decoder(output, hidden, cell)
            outputs[t] = output
            teacher_force = random.random() < teacher_forcing_ratio
            top1 = output.max(1)[1]
            output = (trg[t] if teacher_force else top1)

        return outputs


In [22]:
INPUT_DIM = len(question_vocab)
print(INPUT_DIM)
OUTPUT_DIM = len(answer_vocab)
print(OUTPUT_DIM)
# ENC_EMB_DIM = 256
# DEC_EMB_DIM = 256
# ENC_HID_DIM = 512
# DEC_HID_DIM = 512
# ATTN_DIM = 64
# ENC_DROPOUT = 0.5
# DEC_DROPOUT = 0.5

ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
N_LAYERS = 2
HID_DIM = 512
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)

#attn = Attention(ENC_HID_DIM, DEC_HID_DIM, ATTN_DIM)

dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)

model = Seq2Seq(enc, dec, device).to(device)


def init_weights(m: nn.Module):
    for name, param in m.named_parameters():
        if 'weight' in name:
            nn.init.normal_(param.data, mean=0, std=0.01)
        else:
            nn.init.constant_(param.data, 0)


model.apply(init_weights)

optimizer = optim.Adam(model.parameters())


def count_parameters(model: nn.Module):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


print(f'The model has {count_parameters(model):,} trainable parameters')

6266
4781
The model has 12,637,101 trainable parameters


In [23]:
PAD_IDX = question_vocab.get_stoi()['<pad>']
print(PAD_IDX)
criterion = nn.CrossEntropyLoss(ignore_index=PAD_IDX)

1


In [123]:
import math
import time


def train(model: nn.Module,
          iterator: torch.utils.data.DataLoader,
          optimizer: optim.Optimizer,
          criterion: nn.Module,
          clip: float):

    model.train()

    epoch_loss = 0

    for _, (src, trg) in enumerate(iterator):
        src, trg = src.to(device), trg.to(device)

        optimizer.zero_grad()

        output = model(src, trg)

        output = output[1:].view(-1, output.shape[-1])
        trg = trg[1:].view(-1)

        loss = criterion(output, trg)

        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)

        optimizer.step()

        epoch_loss += loss.item()

    return epoch_loss / len(iterator)


def evaluate(model: nn.Module,
             iterator: torch.utils.data.DataLoader,
             criterion: nn.Module):

    model.eval()

    epoch_loss = 0

    with torch.no_grad():

        for _, (src, trg) in enumerate(iterator):
            src, trg = src.to(device), trg.to(device)
            output = model(src, trg, 0) #turn off teacher forcing

            output = output[1:].view(-1, output.shape[-1])
            trg = trg[1:].view(-1)

            loss = criterion(output, trg)

            epoch_loss += loss.item()

    return epoch_loss / len(iterator)


def epoch_time(start_time: int,
               end_time: int):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time / 60)
    elapsed_secs = int(elapsed_time - (elapsed_mins * 60))
    return elapsed_mins, elapsed_secs


N_EPOCHS = 65
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):

    start_time = time.time()

    train_loss = train(model, train_iter, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iter, criterion)

    end_time = time.time()

    epoch_mins, epoch_secs = epoch_time(start_time, end_time)

    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'QA-model_04_22_2023_65_EPOCHS.pt')

#test_loss = evaluate(model, test_iter, criterion)

#print(f'| Test Loss: {test_loss:.3f} | Test PPL: {math.exp(test_loss):7.3f} |')

Epoch: 01 | Time: 0m 16s
	Train Loss: 4.148 | Train PPL:  63.332
	 Val. Loss: 9.658 |  Val. PPL: 15645.362
Epoch: 02 | Time: 0m 15s
	Train Loss: 4.051 | Train PPL:  57.427
	 Val. Loss: 9.675 |  Val. PPL: 15917.444
Epoch: 03 | Time: 0m 15s
	Train Loss: 4.035 | Train PPL:  56.539
	 Val. Loss: 9.627 |  Val. PPL: 15173.870
Epoch: 04 | Time: 0m 15s
	Train Loss: 4.031 | Train PPL:  56.340
	 Val. Loss: 9.558 |  Val. PPL: 14153.481
Epoch: 05 | Time: 0m 15s
	Train Loss: 3.943 | Train PPL:  51.558
	 Val. Loss: 9.706 |  Val. PPL: 16416.620
Epoch: 06 | Time: 0m 15s
	Train Loss: 3.869 | Train PPL:  47.877
	 Val. Loss: 9.828 |  Val. PPL: 18554.628
Epoch: 07 | Time: 0m 15s
	Train Loss: 3.861 | Train PPL:  47.513
	 Val. Loss: 9.818 |  Val. PPL: 18362.233
Epoch: 08 | Time: 0m 15s
	Train Loss: 3.746 | Train PPL:  42.340
	 Val. Loss: 9.897 |  Val. PPL: 19876.090
Epoch: 09 | Time: 0m 15s
	Train Loss: 3.801 | Train PPL:  44.766
	 Val. Loss: 9.957 |  Val. PPL: 21094.882
Epoch: 10 | Time: 0m 15s
	Train Loss:

In [ ]:
model.load_state_dict(torch.load('QA-model_04_22_2023_65_EPOCHS.pt'))

In [124]:

def chat_evaluate(model: nn.Module,
             iterator: torch.utils.data.DataLoader,
             criterion: nn.Module):

    model.eval()

    epoch_loss = 0

    with torch.no_grad():

        for _, (src, trg) in enumerate(iterator):
            src, trg = src.to(device), trg.to(device)
            output = model(src, trg, 0) #turn off teacher forcing
            output = output[1:].view(-1, output.shape[-1])
            trg = trg[1:].view(-1)
            loss = criterion(output, trg)

            epoch_loss += loss.item()
            result = []
            for tensor in output:
                _, top_token = tensor.data.topk(1)
                if top_token.item() == 1:
                    break
                else:
                    word = answer_vocab.get_itos()[top_token.item()]
                    result.append(word)     
            return  result


def generate_test_batch(data_batch):
    de_batch, en_batch = [], []
    for (de_item, en_item) in data_batch:
        de_batch.append(torch.cat([torch.tensor([BOS_IDX]), de_item, torch.tensor([EOS_IDX])], dim=0))
        en_batch.append(torch.cat([torch.tensor([BOS_IDX]), en_item, torch.tensor([EOS_IDX])], dim=0))
        de = pad_sequence(de_batch, padding_value=PAD_IDX)
        en = pad_sequence(en_batch, padding_value=PAD_IDX)
    return de, en


In [125]:
pd.set_option('display.max_rows', None)
dt_test = df_train.sample(n = 1)
#dt_test
dt_test_1 = dt_test.head(1)
print(dt_test_1)
test_data = data_process(dt_test_1)
test_iter = DataLoader(test_data, batch_size=BATCH_SIZE,
                       shuffle=True, collate_fn=generate_test_batch)

next(iter(test_iter))

result = chat_evaluate(model, test_iter, criterion)
print('Answer:', result)

                                               question answers
4148  If someone serves three consecutive terms as m...    four
Answer: ['four', '<eos>']


In [126]:
pd.set_option('display.max_rows', None)
dt_test = df_train.sample(n = 1)
#dt_test
dt_test_1 = dt_test.head(1)
print(dt_test_1)
test_data = data_process(dt_test_1)
test_iter = DataLoader(test_data, batch_size=BATCH_SIZE,
                       shuffle=True, collate_fn=generate_test_batch)

next(iter(test_iter))

result = chat_evaluate(model, test_iter, criterion)
print('Answer:', result)

                                     question   answers
5290  Who has the power to destroy bad karma?  Amitābha
Answer: ['H&M', '<eos>']


In [128]:
pd.set_option('display.max_rows', None)
dt_test = df_train.sample(n = 1)
#dt_test
dt_test_1 = dt_test.head(1)
print(dt_test_1)
test_data = data_process(dt_test_1)
test_iter = DataLoader(test_data, batch_size=BATCH_SIZE,
                       shuffle=True, collate_fn=generate_test_batch)

next(iter(test_iter))

result = chat_evaluate(model, test_iter, criterion)
print('Answer:', result)

                                               question            answers
4528  Over 90% of homes use solar hot water systems ...  Israel and Cyprus
Answer: ['Israel', 'and', 'Cyprus', '<eos>']


In [129]:
pd.set_option('display.max_rows', None)
dt_test = df_train.sample(n = 1)
#dt_test
dt_test_1 = dt_test.head(1)
print(dt_test_1)
test_data = data_process(dt_test_1)
test_iter = DataLoader(test_data, batch_size=BATCH_SIZE,
                       shuffle=True, collate_fn=generate_test_batch)

next(iter(test_iter))

result = chat_evaluate(model, test_iter, criterion)
print('Answer:', result)

                                            question            answers
3001  What were they using the airport to stage for?  relief operations
Answer: ['relief', 'operations', '<eos>']


In [130]:
pd.set_option('display.max_rows', None)
dt_test = df_train.sample(n = 1)
#dt_test
dt_test_1 = dt_test.head(1)
print(dt_test_1)
test_data = data_process(dt_test_1)
test_iter = DataLoader(test_data, batch_size=BATCH_SIZE,
                       shuffle=True, collate_fn=generate_test_batch)

next(iter(test_iter))

result = chat_evaluate(model, test_iter, criterion)
print('Answer:', result)

                                             question           answers
941  While in Berlin he saw the operatic work of who?  Gaspare Spontini
Answer: ['Gaspare', 'Spontini', '<eos>']


In [133]:
pd.set_option('display.max_rows', None)
dt_test = df_train.sample(n = 1)
#dt_test
dt_test_1 = dt_test.head(1)
print(dt_test_1)
test_data = data_process(dt_test_1)
test_iter = DataLoader(test_data, batch_size=BATCH_SIZE,
                       shuffle=True, collate_fn=generate_test_batch)

next(iter(test_iter))

result = chat_evaluate(model, test_iter, criterion)
print('Answer:', result)

                                               question     answers
1356  What was the name under scholarly publications...  Paderewski
Answer: ['Paderewski', '<eos>']


In [134]:
pd.set_option('display.max_rows', None)
dt_test = df_train.sample(n = 1)
#dt_test
dt_test_1 = dt_test.head(1)
print(dt_test_1)
test_data = data_process(dt_test_1)
test_iter = DataLoader(test_data, batch_size=BATCH_SIZE,
                       shuffle=True, collate_fn=generate_test_batch)

next(iter(test_iter))

result = chat_evaluate(model, test_iter, criterion)
print('Answer:', result)

                                               question answers
4530  What country is the leader in the implementati...   China
Answer: ['Manhattan', '<eos>']


In [137]:
pd.set_option('display.max_rows', None)
dt_test = df_train.sample(n = 1)
#dt_test
dt_test_1 = dt_test.head(1)
print(dt_test_1)
test_data = data_process(dt_test_1)
test_iter = DataLoader(test_data, batch_size=BATCH_SIZE,
                       shuffle=True, collate_fn=generate_test_batch)

next(iter(test_iter))

result = chat_evaluate(model, test_iter, criterion)
print('Answer:', result)

                                       question       answers
2947  What was the predicted time of the quake?  around May 8
Answer: ['New', '<eos>', '<eos>', 'Week']


In [138]:
pd.set_option('display.max_rows', None)
dt_test = df_train.sample(n = 1)
#dt_test
dt_test_1 = dt_test.head(1)
print(dt_test_1)
test_data = data_process(dt_test_1)
test_iter = DataLoader(test_data, batch_size=BATCH_SIZE,
                       shuffle=True, collate_fn=generate_test_batch)

next(iter(test_iter))

result = chat_evaluate(model, test_iter, criterion)
print('Answer:', result)

                                               question       answers
1917  What entity did Universal have a contract with...  iTunes Store
Answer: ['iTunes', 'Store', '<eos>']


In [145]:
'''Reference:-  https://pytorch.org/tutorials/beginner/torchtext_translation_tutorial.html
https://a323d4c3b06f40ed9ea707b0af01e94b.udacity-student-workspaces.com/notebooks/seq2seq_translation_tutorial.ipynb
https://github.com/bentrevett/pytorch-seq2seq/blob/master/1%20-%20Sequence%20to%20Sequence%20Learning%20with%20Neural%20Networks.ipynb
Some methods are taken from this tutorial

 Only 5000 records are trained'''



'Reference:-  https://pytorch.org/tutorials/beginner/torchtext_translation_tutorial.html\nhttps://a323d4c3b06f40ed9ea707b0af01e94b.udacity-student-workspaces.com/notebooks/seq2seq_translation_tutorial.ipynb\nhttps://github.com/bentrevett/pytorch-seq2seq/blob/master/1%20-%20Sequence%20to%20Sequence%20Learning%20with%20Neural%20Networks.ipynb\nSome methods are taken from this tutorial\n\n Only 5000 records are trained'